In [1]:
import numpy as np
import nibabel as nib
from util.path import Path

In [4]:
%cd ..

/scratch/gpfs/zzada/fconv


In [2]:
from util import subject

In [8]:
sub_id = 105
condition = 'G'
dft = subject.get_timing(sub_id, condition=condition)
dft2 = dft[["run", "trial", "condition"]].drop_duplicates().dropna()
dft2["trial"] = dft2.trial.astype(int)
rt_dict = dft2[["run", "trial"]].groupby("run")["trial"].apply(list).to_dict()

In [9]:
dft2

,run,trial,condition
2,1,1,G
28,1,4,G
38,2,1,G
58,2,3,G
78,3,2,G
91,3,4,G
103,4,2,G
117,4,4,G
132,5,2,G
147,5,4,G


In [10]:
rt_dict

{1: [1, 4], 2: [1, 3], 3: [2, 4], 4: [2, 4], 5: [2, 4]}

In [6]:
subject.get_trials(5, condition='G')

{1: [1, 4], 2: [1, 3], 3: [2, 4], 4: [2, 4], 5: [2, 4]}

In [6]:
%cd ..

/scratch/gpfs/zzada/fconv


In [35]:
path = Path(root='data/derivatives/fmriprep', sub='004', ses=1, datatype='func', task='Black', run=1, space='T1w', desc='brain_mask', ext='.nii.gz')
path

data/derivatives/fmriprep/sub-004/ses-1/func/sub-004_ses-1_task-Black_run-1_space-T1w_desc-brain_mask.nii.gz

In [36]:
img = nib.load(path)
mask = img.get_fdata().astype(bool)
mask.shape

(48, 59, 49)

In [28]:
path = Path(root='data/derivatives/fmriprep', sub='004', ses=1, datatype='func', task='Black', run=1, space='T1w', desc='preproc_bold', ext='.nii.gz')
path

data/derivatives/fmriprep/sub-004/ses-1/func/sub-004_ses-1_task-Black_run-1_space-T1w_desc-preproc_bold.nii.gz

In [29]:
img = nib.load(path)
data = img.get_fdata()
data.shape

(48, 59, 49, 550)

In [39]:
data[mask, ...].shape

(61997, 550)